In [1]:
#importer les librairies
import pandas as pd
import re


# création de la fonction qui permet de récupérer les tables dans une requête sql

In [2]:
# La requête SQL de test
sql_query = """
SELECT Clients.Nom, Clients.Prenom, Clients.Email, SUM(Produits.Prix * DétailsCommande.Quantité) AS MontantTotal
FROM Clients
JOIN Commandes ON Clients.IdClient = Commandes.IdClient
JOIN DétailsCommande ON Commandes.IdCommande = DétailsCommande.IdCommande
JOIN Produits ON DétailsCommande.IdProduit = Produits.IdProduit
WHERE Commandes.DateCommande >= DATE_SUB(NOW(), INTERVAL 1 MONTH)
GROUP BY Clients.IdClient
HAVING MontantTotal >= 100;
"""

# Expression régulière pour extraire les noms de table
pattern = r'\b(?:FROM|JOIN)\s+(\w+)\b'

# Recherche des correspondances dans la requête SQL
table = re.findall(pattern, sql_query, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)


Tables utilisées dans la requête SQL:
['Clients', 'Commandes', 'DétailsCommande', 'Produits']


## Test avec différentes requêtes

In [3]:
sql1="select * from (select nom, adresse from Immeuble where id=1) as Koudalou"

table = re.findall(pattern, sql1, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)

Tables utilisées dans la requête SQL:
['Immeuble']


In [4]:
sql="SELECT * FROM Table"

table = re.findall(pattern, sql, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)

Tables utilisées dans la requête SQL:
['Table']


In [5]:
sql3="SELECT * FROM question WHERE q_id IN (SELECT r_fk_question_id FROM reponse WHERE r_date_ajout BETWEEN '2013-01-01' AND '2013-12-31')"

table = re.findall(pattern, sql3, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)

Tables utilisées dans la requête SQL:
['question', 'reponse']


In [6]:
sql4="SELECT sum(a.Population)FROM travail.communes64 AS a WHERE EXISTS(SELECT * FROM travail.communes64 AS b WHERE b.Population > 10)"

table = re.findall(pattern, sql4, re.IGNORECASE)

# Affichage des tables extraites
print("Tables utilisées dans la requête SQL:")
print(table)

Tables utilisées dans la requête SQL:
['travail', 'travail']


# Création d'un dataframe pour faire tourner le script sur un fichier csv

In [7]:
#créer le dataframe à partir d'un fichier csv
df=pd.read_csv("sql_requetes.csv", sep=';')

In [8]:
#lire toutes les infos du df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [9]:
#lire les 10 premières lignes du dataframe
df.head(10)

,projet,requete_sql
0,ATP,"select * from (select nom, adresse from Immeuble where id=1) as Koudalou"
1,ATP,SELECT * FROM Table
2,ATP,SELECT * FROM question WHERE q_id IN (SELECT r_fk_question_id FROM reponse WHERE r_date_ajout BETWEEN 2013-01-01 AND 2013-12-31 )
3,ATP,SELECT sum(a.Population)FROM travail.communes64 AS a WHERE EXISTS(SELECT * FROM travail.communes64 AS b WHERE b.Population > 10)
4,ATP,"SELECT timestamp, log_id, severity, json_payload, resource, labels FROM TABLE WHERE timestamp > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 HOUR) AND log_id = cloudaudit.googleapis.com/data_access ORDER BY timestamp ASC LIMIT 100"
5,ATP,SELECT * FROM Orders WHERE CustomerID IN (SELECT CustomerID FROM Customers WHERE Country = 'USA' AND City IN (SELECT DISTINCT ShipCity FROM Shippers WHERE ShipperID IN (SELECT ShipperID FROM OrderDetails WHERE ProductID IN (SELECT ProductID FROM Products WHERE CategoryID = 1))))
6,ATP,"SELECT COUNT(count_table.mois) FROM(SELECT st.mois, st.pays, st.somme_transactions, stm.somme_transactions FROM (\tSELECT EXTRACT(MONTH FROM t.date) AS mois, c.pays, SUM(t.montant) AS somme_transactions\t\tFROM transaction t\tINNER JOIN city cON t.ville = c.villeGROUP BY mois, c.pays) stINNER JOIN somme_transactions_mois stm ON st.mois = stm.mois AND st.pays = stm.pays WHERE st.somme_transactions != stm.somme_transactions ORDER BY st.mois ASC, st.pays ASC) AS count_table GROUP BY count_table.mois"


In [10]:
def extraire_table(sql_query):
    # Expression régulière pour extraire les noms de table
    pattern = r'\b(?:FROM|JOIN)\s+(\w+)\b'

    # Recherche des correspondances dans la requête SQL
    table = re.findall(pattern, sql_query, re.IGNORECASE)
    return table 

# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df['table'] = df['requete_sql'].apply(extraire_table)

# S'assurer que la colonne 'table' est de type chaîne
df['table'] = df['table'].astype(str)

# Supprimer les crochets de la colonne 'table'
df['table']=df['table'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df['table'] = df['table'].str.replace('\'', '')

# Afficher le DataFrame résultant
df.head(10)

,projet,requete_sql,table
0,ATP,"select * from (select nom, adresse from Immeuble where id=1) as Koudalou",Immeuble
1,ATP,SELECT * FROM Table,Table
2,ATP,SELECT * FROM question WHERE q_id IN (SELECT r_fk_question_id FROM reponse WHERE r_date_ajout BETWEEN 2013-01-01 AND 2013-12-31 ),"question, reponse"
3,ATP,SELECT sum(a.Population)FROM travail.communes64 AS a WHERE EXISTS(SELECT * FROM travail.communes64 AS b WHERE b.Population > 10),"travail, travail"
4,ATP,"SELECT timestamp, log_id, severity, json_payload, resource, labels FROM TABLE WHERE timestamp > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 HOUR) AND log_id = cloudaudit.googleapis.com/data_access ORDER BY timestamp ASC LIMIT 100",TABLE
5,ATP,SELECT * FROM Orders WHERE CustomerID IN (SELECT CustomerID FROM Customers WHERE Country = 'USA' AND City IN (SELECT DISTINCT ShipCity FROM Shippers WHERE ShipperID IN (SELECT ShipperID FROM OrderDetails WHERE ProductID IN (SELECT ProductID FROM Products WHERE CategoryID = 1)))),"Orders, Customers, Shippers, OrderDetails, Products"
6,ATP,"SELECT COUNT(count_table.mois) FROM(SELECT st.mois, st.pays, st.somme_transactions, stm.somme_transactions FROM (\tSELECT EXTRACT(MONTH FROM t.date) AS mois, c.pays, SUM(t.montant) AS somme_transactions\t\tFROM transaction t\tINNER JOIN city cON t.ville = c.villeGROUP BY mois, c.pays) stINNER JOIN somme_transactions_mois stm ON st.mois = stm.mois AND st.pays = stm.pays WHERE st.somme_transactions != stm.somme_transactions ORDER BY st.mois ASC, st.pays ASC) AS count_table GROUP BY count_table.mois","t, transaction, city, somme_transactions_mois"


In [11]:
df.to_csv("table_sql.csv")

In [12]:
#test avec une extraction depuis Talend de la table job_sql_table
#erreur car les requetes se mettent sur plusieurs lignes...

df1=pd.read_csv("out.csv", sep=(","))
df1.head()

ParserError: Error tokenizing data. C error: Expected 2 fields in line 10605, saw 10
